#### Note:

1. 進入點：verdict-cls-debug/uie
2. 標好的 label studio output 放在 ./label_studio_data 內

##### step 1
把 label studio output 轉成 Doccano 格式

In [ ]:
python labelstudio2doccano.py --labelstudio_file ./label_studio_data/uie_for_utc_general_plus.json                                                         

##### step 2
切 step 1 的 output 切成 train/dev/test

In [ ]:
python doccano.py

##### step 3
開train

跑越久不一定越好，模型可能反而容易預測空值

最好的checkpoint怎麼train 我忘了

In [ ]:
python finetune.py  \
    --device gpu \
    --logging_steps 10 \
    --save_steps 100 \
    --eval_steps 100 \
    --seed 1000 \
    --model_name_or_path uie-base \
    --output_dir ./checkpoint/model_best \
    --train_path data/train.txt \
    --dev_path data/dev.txt  \
    --max_seq_length 512  \
    --per_device_eval_batch_size 16 \
    --per_device_train_batch_size  16 \
    --num_train_epochs 30  \
    --learning_rate 5e-6 \
    --label_names "start_positions" "end_positions" \
    --do_train \
    --do_eval \
    --do_export \
    --export_model_dir ./checkpoint/model_best \
    --overwrite_output_dir \
    --disable_tqdm True \
    --metric_for_best_model eval_recall \
    --load_best_model_at_end  True \
    --save_total_limit 1 

Eval

In [ ]:
python evaluate.py --model_path checkpoint/model_best/checkpoint-700 --test_path data/dev.txt

實驗看看模型的擷取能力如何

In [ ]:
python filter_text.py --model_name_or_path checkpoint/model_best/ --index_of_toy_data 0

正式用 UIE 擷取文本 （下面這段可以不用跑）

In [ ]:
python uie_preprocessing.py --max_seq_len 768 --threshold 0.0 --uie_model_name_or_path ../../uie/checkpoint/use_checkpoint/checkpoint-700 --out_folder_name uie_30_data_768_2